<a href="https://colab.research.google.com/github/maishatabassum2/Titanic-Dataset/blob/main/feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opendatasets


In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/c/titanic/data?utm_source=chatgpt.com")

Skipping, found downloaded files in "./titanic" (use force=True to force download)


In [ ]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier



In [ ]:
import pandas as pd
titanicdata = pd.read_csv("/content/titanic/train.csv")

features


In [ ]:
titanicdata['Title'] = titanicdata['Name'].apply(lambda name: name.split(',')[1].split('.')[0].strip())


normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir":        "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess": "Royalty",
    "Countess":   "Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Miss",
    "Mr":         "Mr",
    "Mrs":        "Mrs",
    "Miss":       "Miss",
    "Master":     "Master",
    "Lady":       "Royalty"
}

titanicdata['Title'] = titanicdata['Title'].map(normalized_titles)


print(titanicdata['Title'].value_counts())


Title
Mr         517
Miss       185
Mrs        126
Master      40
Officer     18
Royalty      5
Name: count, dtype: int64


family size feature

In [ ]:
titanicdata['FamilySize'] = titanicdata['SibSp'] + titanicdata['Parch'] + 1


cabin feature

In [ ]:
titanicdata['CabinLetter'] = titanicdata['Cabin'].map(lambda x: str(x)[0] if pd.notnull(x) else 'Unknown')

isAlone

In [ ]:
titanicdata['IsAlone'] = (titanicdata['FamilySize'] == 1).astype(int)


age imputer


In [ ]:
titanicdata['Age'] = titanicdata['Age'].fillna(titanicdata['Age'].median())

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_cols = ['Sex', 'Title', 'CabinLetter', 'Embarked']

titanic_encoded = pd.get_dummies(titanicdata, columns=cat_cols, drop_first=True)

X = titanic_encoded.drop('Survived', axis=1)
y = titanic_encoded['Survived']


In [ ]:
X = titanicdata.drop(columns=['Survived', 'Name', 'Ticket', 'Cabin', 'PassengerId'])
y = titanicdata['Survived']

cat_cols = ['Sex', 'Title', 'CabinLetter', 'Embarked']
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True)

X_encoded = X_encoded.fillna(X_encoded.median())


from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rfe = RFE(rf, n_features_to_select=5)
rfe.fit(X_encoded, y)

rfe_ranking = pd.Series(rfe.ranking_, index=X_encoded.columns).sort_values()
print("RFE feature ranking (1 = most important):")
print(rfe_ranking)

RFE feature ranking (1 = most important):
Pclass                  1
Age                     1
Sex_male                1
Fare                    1
Title_Mr                1
FamilySize              2
CabinLetter_Unknown     3
SibSp                   4
Title_Miss              5
Title_Mrs               6
Parch                   7
Embarked_S              8
IsAlone                 9
Embarked_Q             10
CabinLetter_E          11
CabinLetter_C          12
Title_Officer          13
CabinLetter_B          14
CabinLetter_D          15
CabinLetter_F          16
CabinLetter_G          17
Title_Royalty          18
CabinLetter_T          19
dtype: int64


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif




selector = SelectKBest(score_func=f_classif, k=6)
selector.fit(X_encoded, y)

# Get scores
scores = pd.Series(selector.scores_, index=X_encoded.columns).sort_values(ascending=False)
print("SelectKBest feature scores:")
print(scores.head(10))

selected_features = X_encoded.columns[selector.get_support()]

print(selected_features)

SelectKBest feature scores:
Title_Mr               383.945495
Sex_male               372.405724
Title_Mrs              117.749079
Pclass                 115.031272
Title_Miss             112.860827
CabinLetter_Unknown     99.253142
Fare                    63.030764
IsAlone                 38.353651
CabinLetter_B           28.117228
Embarked_S              22.075469
dtype: float64
Index(['Pclass', 'Sex_male', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
       'CabinLetter_Unknown'],
      dtype='object')
